In [1]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [63]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-49969")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Software Engineer -III, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer -III, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, Inc.

In [68]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [69]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer -III, ITC',
 'experience': '5+ years of hands-on industry software development experience',
 'skills': ['Front-end frameworks like React, Angular, or Vue.js',
  'Cloud architecture, modern DevOps, infrastructure as code, CI/CD and related tools',
  'AWS products including Lambda, Step Functions, DynamoDB, Elasticsearch, s3',
  'Modern testing methodologies and frameworks such as Mocha, Jasmine and Jest',
  'Architectural design patterns and computer-science fundamentals',
  'Implementing and integrating AI, Machine Learning and related data solutions',
  'Modern software architectural principles and patterns (REST, domain-driven design, microservices, etc)'],
 'description': 'We’re looking for a Senior Software Engineer to solve complex software engineering problems supporting Nike’s pursuit of delivering state of the art tools to our product developers and broader creation community.'}

In [70]:
type(json_res)

dict

In [71]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [72]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [77]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links


[[{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/java-portfolio'}]]

In [74]:
job

[{'role': 'Administrative Support',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Advanced Innovation',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Air Manufacturing Innovation',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Aviation', 'experience': None, 'skills': None, 'description': None},
 {'role': 'Communications',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Customer Service',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Design', 'experience': None, 'skills': None, 'description': None},
 {'role': 'Digital', 'experience': None, 'skills': None, 'description': None},
 {'role': 'Facilities',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Finance & Accounting',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Government & Public Affairs',
  'experience': None,
  'skills': None,
  'desc

In [75]:
job = json_res
job['skills']

['Front-end frameworks like React, Angular, or Vue.js',
 'Cloud architecture, modern DevOps, infrastructure as code, CI/CD and related tools',
 'AWS products including Lambda, Step Functions, DynamoDB, Elasticsearch, s3',
 'Modern testing methodologies and frameworks such as Mocha, Jasmine and Jest',
 'Architectural design patterns and computer-science fundamentals',
 'Implementing and integrating AI, Machine Learning and related data solutions',
 'Modern software architectural principles and patterns (REST, domain-driven design, microservices, etc)']

In [76]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Solutions for Nike's ITC Team

Dear Hiring Manager,

I came across the job description for a Software Engineer - III, ITC at Nike, and I'm excited to introduce AtliQ, an AI & Software Consulting company that can help fulfill your team's needs. With our expertise in software development, we're confident that we can provide the necessary support to deliver state-of-the-art tools to your product developers and broader creation community.

At AtliQ, we have a strong portfolio of projects that demonstrate our capabilities in software engineering. Our team has hands-on experience with front-end frameworks like React, which is showcased in our portfolio: https://example.com/react-portfolio. We also have expertise in cloud architecture, modern DevOps, and infrastructure as code, with experience working with AWS products such as Lambda, Step Functions, DynamoDB, Elasticsearch, and s3.

Our team is well-versed in modern testing methodologies and frameworks li